In [4]:
import os
import shutil
import random
from pathlib import Path
from collections import defaultdict

In [5]:
def stratified_sample_and_copy(input_root="policies", output_root="sampled_policies", sample_targets=None):
    """
    Stratified sampling and copying of policy files per tool across repositories.
    """
    if sample_targets is None:
        raise ValueError("You must provide a dictionary of sample targets per tool.")
    
    os.makedirs(output_root, exist_ok=True)

    for tool_name, target_count in sample_targets.items():
        print(f"\n📂 Processing {tool_name} ({target_count} files)...")
        tool_path = os.path.join(input_root, tool_name)
        if not os.path.isdir(tool_path):
            print(f"⚠️  Folder for tool '{tool_name}' not found.")
            continue

        # Collect all files across all repos for this tool
        all_files = []
        repo_files = defaultdict(list)
        for repo in os.listdir(tool_path):
            repo_path = os.path.join(tool_path, repo)
            if os.path.isdir(repo_path):
                for root, _, files in os.walk(repo_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        all_files.append(file_path)
                        repo_files[repo].append(file_path)

        if len(all_files) == 0:
            print(f"⚠️  No files found for {tool_name}.")
            continue

        print(f"🔍 Found {len(all_files)} files across {len(repo_files)} repositories.")

        # Sample strategy: interleave across repositories
        selected_files = []
        repo_cycle = list(repo_files.keys())
        while len(selected_files) < target_count and repo_cycle:
            random.shuffle(repo_cycle)
            for repo in repo_cycle:
                if repo_files[repo]:
                    selected_files.append(repo_files[repo].pop())
                    if len(selected_files) >= target_count:
                        break
            repo_cycle = [r for r in repo_cycle if repo_files[r]]  # Keep repos with remaining files

        print(f"✅ Selected {len(selected_files)} files for {tool_name}.")

        # Copy to destination
        dest_folder = os.path.join(output_root, tool_name)
        os.makedirs(dest_folder, exist_ok=True)

        for i, src_path in enumerate(selected_files, start=1):
            ext = Path(src_path).suffix
            dest_file = os.path.join(dest_folder, f"{tool_name}_{i}{ext}")
            shutil.copy2(src_path, dest_file)

        print(f"📥 Copied {len(selected_files)} files to {dest_folder}")

In [6]:
# Example usage:
sample_targets = {
    "HashiCorp Sentinel": 15,
    "Open Policy Agent (OPA)": 234,  # Rego
    "Pulumi": 17,
    "Cedar Policy Language (CPL)": 2,
    "Kyverno OSS": 62,
    "Cloud Custodian": 11,
    "AWS Config": 11,
    "OpagateKeeper": 26,
    "Kubewarden": 1
}

stratified_sample_and_copy(
    input_root="policies",
    output_root="sampled_policies",
    sample_targets=sample_targets
)


📂 Processing HashiCorp Sentinel (15 files)...
🔍 Found 514 files across 22 repositories.
✅ Selected 15 files for HashiCorp Sentinel.
📥 Copied 15 files to sampled_policies\HashiCorp Sentinel

📂 Processing Open Policy Agent (OPA) (234 files)...
🔍 Found 7959 files across 137 repositories.
✅ Selected 234 files for Open Policy Agent (OPA).
📥 Copied 234 files to sampled_policies\Open Policy Agent (OPA)

📂 Processing Pulumi (17 files)...
🔍 Found 369 files across 17 repositories.
✅ Selected 17 files for Pulumi.
📥 Copied 17 files to sampled_policies\Pulumi

📂 Processing Cedar Policy Language (CPL) (2 files)...
🔍 Found 89 files across 11 repositories.
✅ Selected 2 files for Cedar Policy Language (CPL).
📥 Copied 2 files to sampled_policies\Cedar Policy Language (CPL)

📂 Processing Kyverno OSS (62 files)...
🔍 Found 1188 files across 87 repositories.
✅ Selected 62 files for Kyverno OSS.
📥 Copied 62 files to sampled_policies\Kyverno OSS

📂 Processing Cloud Custodian (11 files)...
🔍 Found 457 files a